In [2]:
# import pandas as pd
# import requests
# from time import sleep
# from tqdm import tqdm

# def search_wikipedia_api(display_name):
#     """
#     Search Wikipedia API for an artist using their display name
#     """
#     search_url = "https://en.wikipedia.org/w/api.php"
    
#     # Construct search query - try with and without "artist" qualifier
#     search_terms = [
#         f"{display_name} artist",
#         f"{display_name} painter",
#         f"{display_name}"
#     ]
    
#     headers = {
#         'User-Agent': 'ArtistResearchBot/1.0 (your@email.com)'
#     }
    
#     for search_term in search_terms:
#         try:
#             params = {
#                 "action": "query",
#                 "format": "json",
#                 "list": "search",
#                 "srsearch": search_term,
#                 "srlimit": 1
#             }
            
#             response = requests.get(search_url, params=params, headers=headers)
#             response.raise_for_status()
#             data = response.json()
            
#             if data["query"]["search"]:
#                 page_id = data["query"]["search"][0]["pageid"]
                
#                 # Get page content
#                 params = {
#                     "action": "query",
#                     "format": "json",
#                     "prop": "extracts|categories",
#                     "exintro": True,
#                     "explaintext": True,
#                     "pageids": page_id
#                 }
                
#                 content_response = requests.get(search_url, params=params, headers=headers)
#                 content_response.raise_for_status()
#                 content_data = content_response.json()
                
#                 page_content = content_data["query"]["pages"][str(page_id)]
                
#                 # Check if the page is about an artist by looking at categories
#                 categories = page_content.get("categories", [])
#                 is_artist = any("artist" in str(cat).lower() or 
#                               "painter" in str(cat).lower() or 
#                               "sculptor" in str(cat).lower() 
#                               for cat in categories)
                
#                 if is_artist:
#                     return {
#                         "extract": page_content["extract"],
#                         "page_id": page_id,
#                         "confidence": "high" if "artist" in search_term else "medium"
#                     }
                
#         except Exception as e:
#             print(f"Error searching for {display_name}: {str(e)}")
#             continue
            
#     return {"extract": "", "page_id": None, "confidence": "none"}

# def process_artists_file(input_file, output_file, sample_size=10):
#     """
#     Process CSV file containing artist information
#     Expected column: DisplayName (in format "Firstname Lastname")
#     """
#     # Read the CSV file and take only the first sample_size rows
#     df = pd.read_csv(input_file).head(sample_size)
    
#     # Create empty lists to store results
#     extracts = []
#     page_ids = []
#     confidence_levels = []
    
#     # Process each artist with a progress bar
#     for _, row in tqdm(df.iterrows(), total=len(df), desc="Processing artists"):
#         result = search_wikipedia_api(row['DisplayName'])
#         extracts.append(result['extract'][:500] if result['extract'] else '')  # Limit to first 500 chars
#         page_ids.append(result['page_id'])
#         confidence_levels.append(result['confidence'])
        
#         # Be nice to Wikipedia's servers
#         sleep(1)
    
#     # Add new columns to the dataframe
#     df['wikipedia_extract'] = extracts
#     df['wikipedia_page_id'] = page_ids
#     df['match_confidence'] = confidence_levels
    
#     # Save the results
#     df.to_csv(output_file, index=False)
#     print(f"Results saved to {output_file}")
    
#     # Print some statistics
#     total = len(df)
#     found = sum(1 for x in page_ids if x is not None)
#     print(f"\nStats:")
#     print(f"Total artists processed: {total}")
#     print(f"Artists found: {found} ({(found/total)*100:.1f}%)")
#     print(f"Artists not found: {total - found} ({((total-found)/total)*100:.1f}%)")
    
#     # Print the actual results for quick review
#     print("\nResults preview:")
#     for _, row in df.iterrows():
#         print(f"\nArtist: {row['DisplayName']}")
#         print(f"Found: {'Yes' if row['wikipedia_page_id'] else 'No'}")
#         if row['wikipedia_extract']:
#             print(f"Extract: {row['wikipedia_extract'][:200]}...")
#         print(f"Confidence: {row['match_confidence']}")

# if __name__ == "__main__":
#     input_file = "female_artists_with_work_counts.csv"
#     output_file = "female_artists_with_work_counts_wiki_sample_TEST.csv"
#     process_artists_file(input_file, output_file, sample_size=10)  # Process only first 10 artists

Processing artists: 100%|██████████| 10/10 [00:15<00:00,  1.54s/it]

Results saved to female_artists_with_work_counts_wiki_sample_TEST.csv

Stats:
Total artists processed: 10
Artists found: 8 (80.0%)
Artists not found: 2 (20.0%)

Results preview:

Artist: Irene Aronson
Found: Yes
Extract: Irene Hilde Aronson (sometimes Anderson or Aronsohn, 1918–1992) was a German-born American painter and printmaker....
Confidence: high

Artist: Ruth Asawa
Found: Yes
Extract: Ruth Aiko Asawa (January 24, 1926 – August 5, 2013) was an American modernist artist known primarily for her abstract looped-wire sculptures inspired by natural and organic forms. In addition to her t...
Confidence: high

Artist: Isidora Aschheim
Found: Yes
Confidence: none

Artist: Geneviève Asse
Found: Yes
Extract: Geneviève Asse (Vannes, France, 24 January 1923 – 11 August 2021) was a French painter. She died in August 2021 at the age of 98. She was awarded the Grand-Cross of the National Order of the Legion of...
Confidence: high

Artist: Dana Atchley
Found: Yes
Extract: The Federal Art Projec

In [4]:
import pandas as pd
import requests
from time import sleep
from tqdm import tqdm

def search_wikipedia_api(display_name):
    """
    Search Wikipedia API for an artist using their display name
    Now also extracts occupation and years active information
    """
    search_url = "https://en.wikipedia.org/w/api.php"
    
    search_terms = [
        f"{display_name} artist",
        f"{display_name} painter",
        f"{display_name}"
    ]
    
    headers = {
        'User-Agent': 'ArtistResearchBot/1.0 (your@email.com)'
    }
    
    for search_term in search_terms:
        try:
            params = {
                "action": "query",
                "format": "json",
                "list": "search",
                "srsearch": search_term,
                "srlimit": 1
            }
            
            response = requests.get(search_url, params=params, headers=headers)
            response.raise_for_status()
            data = response.json()
            
            if data["query"]["search"]:
                page_id = data["query"]["search"][0]["pageid"]
                title = data["query"]["search"][0]["title"]
                
                # Get page content with additional props
                params = {
                    "action": "query",
                    "format": "json",
                    "prop": "extracts|categories|revisions",
                    "exintro": True,
                    "explaintext": True,
                    "rvprop": "content",
                    "rvslots": "*",
                    "pageids": page_id
                }
                
                content_response = requests.get(search_url, params=params, headers=headers)
                content_response.raise_for_status()
                content_data = content_response.json()
                
                page_content = content_data["query"]["pages"][str(page_id)]
                
                # Check if the page is about an artist
                categories = page_content.get("categories", [])
                is_artist = any("artist" in str(cat).lower() or 
                              "painter" in str(cat).lower() or 
                              "sculptor" in str(cat).lower() 
                              for cat in categories)
                
                if is_artist:
                    # Create Wikipedia URL
                    wiki_url = f"https://en.wikipedia.org/wiki/{title.replace(' ', '_')}"
                    
                    # Try to extract occupation and years active from the page content
                    raw_content = page_content.get("revisions", [{}])[0].get("slots", {}).get("main", {}).get("*", "")
                    
                    # Extract occupation
                    occupation = ""
                    if "occupation" in raw_content.lower():
                        # Look for common occupation field formats
                        occupation_markers = ["| occupation", "|occupation", "| occupation(s)", "|occupation(s)"]
                        for marker in occupation_markers:
                            if marker in raw_content:
                                try:
                                    occupation = raw_content.split(marker)[1].split("\n")[0].split("|")[0].strip(" =[]{}|")
                                    break
                                except:
                                    continue
                    
                    # Extract years active
                    years_active = ""
                    if "years active" in raw_content.lower():
                        years_markers = ["| years active", "|years active", "| years_active", "|years_active"]
                        for marker in years_markers:
                            if marker in raw_content:
                                try:
                                    years_active = raw_content.split(marker)[1].split("\n")[0].split("|")[0].strip(" =[]{}|")
                                    break
                                except:
                                    continue
                    
                    return {
                        "extract": page_content["extract"],
                        "page_id": page_id,
                        "wiki_url": wiki_url,
                        "confidence": "high" if "artist" in search_term else "medium",
                        "occupation": occupation,
                        "years_active": years_active
                    }
                
        except Exception as e:
            print(f"Error searching for {display_name}: {str(e)}")
            continue
            
    return {
        "extract": "",
        "page_id": None,
        "wiki_url": "",
        "confidence": "none",
        "occupation": "",
        "years_active": ""
    }

def process_artists_file(input_file, output_file, sample_size=10, preview_length=150):
    """
    Process CSV file containing artist information
    """
    df = pd.read_csv(input_file).head(sample_size)
    
    extracts = []
    page_ids = []
    wiki_urls = []
    confidence_levels = []
    occupations = []
    years_active = []
    
    for _, row in tqdm(df.iterrows(), total=len(df), desc="Processing artists"):
        result = search_wikipedia_api(row['DisplayName'])
        
        preview = result['extract'][:preview_length] + "..." if result['extract'] else ''
        extracts.append(preview)
        page_ids.append(result['page_id'])
        wiki_urls.append(result['wiki_url'])
        confidence_levels.append(result['confidence'])
        occupations.append(result['occupation'])
        years_active.append(result['years_active'])
        
        sleep(1)
    
    df['wikipedia_preview'] = extracts
    df['wikipedia_page_id'] = page_ids
    df['wikipedia_url'] = wiki_urls
    df['match_confidence'] = confidence_levels
    df['wiki_occupation'] = occupations
    df['wiki_years_active'] = years_active
    
    df.to_csv(output_file, index=False)
    print(f"Results saved to {output_file}")
    
    # Print preview of results
    print("\nResults preview:")
    for _, row in df.iterrows():
        print(f"\nArtist: {row['DisplayName']}")
        print(f"Found: {'Yes' if row['wikipedia_page_id'] else 'No'}")
        if row['wikipedia_preview']:
            print(f"Preview: {row['wikipedia_preview']}")
            print(f"Full article: {row['wikipedia_url']}")
            print(f"Occupation: {row['wiki_occupation']}")
            print(f"Years active: {row['wiki_years_active']}")
        print(f"Confidence: {row['match_confidence']}")

if __name__ == "__main__":
    input_file = "female_artists_with_work_counts.csv"
    output_file = "female_artists_with_work_counts_wiki_years_occupation_TEST.csv"
    process_artists_file(input_file, output_file, sample_size=10, preview_length=500)

Processing artists: 100%|██████████| 10/10 [00:16<00:00,  1.61s/it]

Results saved to female_artists_with_work_counts_wiki_years_occupation_TEST.csv

Results preview:

Artist: Irene Aronson
Found: Yes
Preview: Irene Hilde Aronson (sometimes Anderson or Aronsohn, 1918–1992) was a German-born American painter and printmaker....
Full article: https://en.wikipedia.org/wiki/Irene_Aronson
Occupation: 
Years active: 
Confidence: high

Artist: Ruth Asawa
Found: Yes
Preview: Ruth Aiko Asawa (January 24, 1926 – August 5, 2013) was an American modernist artist known primarily for her abstract looped-wire sculptures inspired by natural and organic forms. In addition to her three-dimensional work, Asawa created an extensive body of works on paper, including abstract and figurative drawings and prints influenced by nature, particularly flowers and plants, and her immediate surroundings.
Born in Norwalk, California in 1926, Asawa was the fourth of seven children born to J...
Full article: https://en.wikipedia.org/wiki/Ruth_Asawa
Occupation: 
Years active: 
Confidence: